In [43]:
import torch.cuda as cuda
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.optim as optim

In [3]:
if cuda.device_count() > 0:
    print('Using GPU')
    cuda.device(0)
else:
    print('Using CPU')

Using CPU


In [5]:
data = [("me gusta comer en la cafeteria".split(), "SPANISH"),
        ("Give it to me".split(), "ENGLISH"),
        ("No creo que sea una buena idea".split(), "SPANISH"),
        ("No it is not a good idea to get lost at sea".split(), "ENGLISH")]

test_data = [("Yo creo que si".split(), "SPANISH"),
             ("it is lost on me".split(), "ENGLISH")]

# word_to_ix maps each word in the vocab to a unique integer, which will be its
# index into the Bag of words vector
word_to_ix = {}
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2

{'me': 0, 'gusta': 1, 'comer': 2, 'en': 3, 'la': 4, 'cafeteria': 5, 'Give': 6, 'it': 7, 'to': 8, 'No': 9, 'creo': 10, 'que': 11, 'sea': 12, 'una': 13, 'buena': 14, 'idea': 15, 'is': 16, 'not': 17, 'a': 18, 'good': 19, 'get': 20, 'lost': 21, 'at': 22, 'Yo': 23, 'si': 24, 'on': 25}


In [28]:
class BoWClassifier(nn.Module):
    def __init__(self, num_labels, vocab_size):
        super(BoWClassifier, self).__init__()
        self.linear = nn.Linear(vocab_size, num_labels)
    def forward(self, bow_vec):
        return  F.log_softmax(self.linear(bow_vec), dim=1)

In [29]:
def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        vec[word_to_ix[word]] += 1
    return vec.view(1, -1)

In [30]:
def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])

In [31]:
model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)

In [32]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[-0.0540,  0.1432,  0.0686,  0.0356, -0.1724, -0.0860, -0.0053,
         -0.0313,  0.0001,  0.0013,  0.0333,  0.0353,  0.1514, -0.0162,
         -0.0849, -0.1953, -0.1726,  0.1206,  0.0512,  0.1351,  0.1380,
          0.0174, -0.1593,  0.1762,  0.1623, -0.0749],
        [ 0.1078,  0.0568, -0.1602,  0.0979,  0.1709,  0.0801, -0.0167,
          0.1325,  0.0465, -0.0307, -0.0091, -0.0215,  0.0377,  0.1660,
          0.0255, -0.1809,  0.0380, -0.1703, -0.0252,  0.1922, -0.1113,
         -0.0014, -0.1416,  0.0693, -0.1323,  0.1354]])
Parameter containing:
tensor([-0.0347, -0.1951])


In [36]:
with torch.no_grad():
    sample = data[0]
    bow_vector = make_bow_vector(sample[0], word_to_ix)
    log_probs = model(bow_vector)
    print(log_probs)

tensor([[-0.8304, -0.5725]])


In [46]:
with torch.no_grad():
    for instance, label in test_data:
        bow_vec = make_bow_vector(instance, word_to_ix)
        log_probs = model(bow_vec)
        print(log_probs)

# Print the matrix column corresponding to "creo"
print(next(model.parameters())[:, word_to_ix["creo"]])


tensor([[-0.4163, -1.0773]])
tensor([[-1.0165, -0.4492]])
tensor(1.00000e-02 *
       [ 3.3342, -0.9101])


In [44]:
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [53]:
label_to_ix = {"SPANISH": 0, "ENGLISH": 1}
for epoch in range(100):
    for instance, label in data:
        model.zero_grad()
        bow_vec = make_bow_vector(instance, word_to_ix)
        target = make_target(label, label_to_ix)
        log_probs = model(bow_vec)
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()

In [52]:
with torch.no_grad():
    for instance, label in test_data:
        bow_vec = make_bow_vector(instance, word_to_ix)
        log_probs = model(bow_vec)
        print(log_probs)
# should be more towards 0 spanish than 1 english
print(next(model.parameters())[:, word_to_ix["creo"]])        

tensor([[-0.0551, -2.9262]])
tensor([[-3.5594, -0.0289]])
tensor([ 0.5270, -0.5028])


https://www.pyimagesearch.com/2016/09/12/softmax-classifiers-explained/

weights should indicate more positive to indicate a particular classification, negative for wrong